In [1]:
#package
import numpy as np
import matplotlib.pyplot as plt
import LorentzDataAnalysisToolbox as DAF

# Data loading and data alignment

In [2]:
#Data Loading & Alignment
FileName=['Data/tek0003.csv',
          'Data/tek0004.csv',
          'Data/tek0005.csv',
          'Data/tek0006.csv',
          'Data/tek0007.csv',
          'Data/tek0008.csv',
          'Data/tek0009.csv',
          'Data/tek0010.csv',
          'Data/tek0011.csv',
          'Data/tek0012.csv']
Voltage=np.zeros([10,100000])
Transmission=np.zeros([10,100000])
for i in range(0,10):
    Data=np.loadtxt(FileName[i],delimiter=',',skiprows=21)
    Voltage[i,:]=Data[:,1]
    Transmission[i,:]=DAF.Normalize_10(Voltage[i,:])
#Cut the data Before alignment
Initial_Cut_bg_index=5000
Initial_Cut_end_index=95000
Transmission=Transmission[:,Initial_Cut_bg_index:Initial_Cut_end_index]
#Data DataAlignment
for i in range(0,9):
    Transmission[i,:],Transmission[i+1,:]=DAF.DataAlignment(Transmission[i,:],Transmission[i+1,:],IfShow=False)

/Users/yumchan/opt/anaconda3/lib/python3.9/site-packages/scipy/signal/_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)


Correlation maximun: k = -2850
Correlation maximun: k = -1526
Correlation maximun: k = -3289
Correlation maximun: k = -1599
Correlation maximun: k = 405
Correlation maximun: k = -5951
Correlation maximun: k = -4976
Correlation maximun: k = -1071
Correlation maximun: k = -1391


# Display the whole data

In [9]:
%matplotlib auto
N_s=10000#Sampling Rate (Sa/s)
v_lam=6#scan velocity (nm/s)
t_total=10#Total time (s)
lam_begin=1520#Beginning wavelength
lam=np.linspace(0,v_lam/N_s*len(Transmission[0,:]),len(Transmission[0,:]))+lam_begin
Label=['0',
       '1',
       '2',
       '3',
       '4',
       '5',
       '6',
       '7',
       '8',
       '9']
#
fig,ax=plt.subplots()
color_list=[]
for i in range(0,10):
    ax.plot(lam,Transmission[i,:],'--*')
ax.legend(Label,loc='lower left')
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Transmission (a.u.)')
plt.show()

Using matplotlib backend: MacOSX


# Local analysis

In [4]:
#Data cutting
Lambda_begin=1530.68
Lambda_end=1530.86

filter_index=lam>=Lambda_begin
New_lam=lam[filter_index]
New_Transmission=Transmission[:,filter_index]
filter_index=New_lam<=Lambda_end
New_lam=New_lam[filter_index]
New_Transmission=New_Transmission[:,filter_index]

In [5]:
#Lorentz line shape fitting
mode_list=[]
mode_Q_list=np.array([])
parameters_list=np.array([0,1,2,3,4,5,6,7,8,9])
for i in range(0,10):
    mode_temp=DAF.Mode((Lambda_begin+Lambda_end)/2,New_Transmission[i,:],New_lam)
    mode_temp.Fitting()
    mode_list.append(mode_temp)
    mode_Q_list=np.append(mode_Q_list,mode_temp.Q)

In [6]:
%matplotlib auto
fig,ax=plt.subplots(1,2,figsize=(18,8))
#parameters vs Q
ax[0].plot(parameters_list,mode_Q_list/1e5,'--*')
ax[0].set_xlabel('Parameters',size=25)
ax[0].set_ylabel('Quality factor Q ($\\times 10^5$)',size=25)
for i in range(0,10):
    ax[1].plot(New_lam,New_Transmission[i,:],'*')
ax[1].legend(Label,loc='lower right')
for i in range(0,10):
    ax[1].plot(New_lam,mode_list[i].Fitting_transimission,'-')
ax[1].set_xlabel('Wavelength (nm)',size=25)
ax[1].set_ylabel('Transmission (a.u.)',size=25)
ax[1].set_xlim([np.min(New_lam),np.max(New_lam)])
ax[1].set_title('Cavity transmission data with Lorentz line shape',size=20)
# plt.savefig('Q.pdf')
plt.show()

Using matplotlib backend: MacOSX
